In [ ]:
test_index = 0

#### testing

In [21]:
from load_data import *

In [22]:
# load_data()

## Loading the data

In [23]:
from load_data import *

In [24]:
X_train,X_test,y_train,y_test = load_data()

4
OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Shine/shine131.jpg
OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Rain/rain141.jpg


/home/indika/Programming/Projects/Python/Artifical-Intelligence/PyTorch/CNN/Weather-Clf/load_data.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.random.shuffle(np.array(DATA))
/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [25]:
len(X_train),len(y_train)

(843, 843)

In [26]:
len(X_test),len(y_test)

(280, 280)

## Test Modelling

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [28]:
class Test_Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.c1 = nn.Conv2d(1,32,5)
        self.c2 = nn.Conv2d(32,64,5)
        self.c3 = nn.Conv2d(64,128,5)
        self.fc4 = nn.Linear(128*10*10,256)
        self.fc5 = nn.Linear(256,4)
    
    def forward(self,X):
        preds = F.max_pool2d(F.relu(self.c1(X)),(2,2))
        preds = F.max_pool2d(F.relu(self.c2(preds)),(2,2))
        preds = F.max_pool2d(F.relu(self.c3(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,128*10*10)
        preds = F.relu(self.fc4(preds))
        preds = self.fc5(preds)
        return preds

In [29]:
device = torch.device('cuda')

In [30]:
BATCH_SIZE = 32

In [31]:
IMG_SIZE = 112

In [32]:
model = Test_Model().to(device)

In [33]:
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [34]:
criterion = nn.CrossEntropyLoss()

In [35]:
EPOCHS = 12

In [36]:
from tqdm import tqdm

In [37]:
PROJECT_NAME = 'Weather-Clf'

In [39]:
import wandb

In [40]:
test_index += 1
wandb.init(project=PROJECT_NAME,name=f'test-{test_index}')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds.to(device)
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item()})
wandb.finish()

  0%|          | 0/12 [00:00<?, ?it/s]/home/indika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
  0%|          | 0/12 [00:00<?, ?it/s]

torch.Size([32, 128, 10, 10])


RuntimeError: 1only batches of spatial targets supported (3D tensors) but got targets of size: : [32]